<a href="https://colab.research.google.com/github/shinisoba/ghg-emission-prediction/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('epa_GHG_econ.csv')
df = pd.DataFrame(data)
df.head()

,ECON_SECTOR,SUBSECTOR,STATE,GHG,Y1990,Y1991,Y1992,Y1993,Y1994,Y1995,...,Y2012,Y2013,Y2014,Y2015,Y2016,Y2017,Y2018,Y2019,Y2020,Y2021
0,Agriculture,Agricultural Soil Management,CA,N2O,1.017179,1.000724,0.996830,1.129317,0.992245,1.164554,...,1.191781,1.200442,1.166076,1.162887,1.049334,1.002140,0.962355,1.069631,1.026559,1.037189
1,Agriculture,Agricultural Soil Management,CA,N2O,0.193156,0.198694,0.210681,0.204771,0.183397,0.214843,...,0.179526,0.145371,0.192600,0.185865,0.191880,0.177998,0.177220,0.197160,0.160450,0.173276
2,Agriculture,Agricultural Soil Management,CA,N2O,4.502394,4.273469,4.211588,4.351497,4.664195,4.578858,...,4.671522,4.881610,4.708510,4.730645,4.274138,3.975159,4.321095,4.363435,4.474570,4.338228
3,Agriculture,Agricultural Soil Management,CA,N2O,2.781138,2.579990,2.690314,2.710979,2.559154,2.748094,...,2.513572,2.422322,2.628997,2.696030,2.669791,2.486140,2.550231,2.748767,2.436414,2.504656
4,Agriculture,Agricultural Soil Management,CO,N2O,0.254721,0.206407,0.236174,0.220155,0.197366,0.351979,...,0.135166,0.168967,0.198379,0.213960,0.196672,0.233065,0.179334,0.198971,0.158968,0.174494


In [ ]:
data.describe(include="object")

,ECON_SECTOR,SUBSECTOR,STATE,GHG
count,22972,22972,22972,22972
unique,8,25,60,21
top,Industry,Fossil Fuel Combustion,CA,CO2
freq,8118,3840,441,9360


In [ ]:
#unique columns
uni_econsec = df.ECON_SECTOR.unique()
print("Economic Sector",uni_econsec)
uni_GHG = df.GHG.unique()
print("GHG",uni_GHG)

Economic Sector ['Agriculture' 'LULUCF Sector Net Total' 'Industry'
 'Electric Power Industry' 'Residential' 'Transportation'
 'U.S. Territories' 'Commercial']
GHG ['N2O' 'CO2' 'CH4' 'HFC-41' 'C4F8O' 'CF4' 'C5F8' 'HFC-32' 'HFC-134a'
 'HFC-23' 'C3F8' 'HFC-143a' 'SF6' 'NF3' 'C4F6' 'C2F6' 'C-C4F8'
 'Unspecified mix of HFCs and PFCs' 'Unspecified mix of PFCs'
 'Unspecified mix of HFCs' 'CO2e']


In [ ]:
df.loc[df['GHG'] == 'CO2e', 'GHG'] = 'CO2'
#extracting only the rows where the GHG is CO2, N2O or CH4
dfn20 = data[data["GHG"] == "N2O"]
dfco2 = data[data["GHG"] == "CO2"]
dfch4 = data[data["GHG"] == "CH4"]
df = pd.concat([dfn20, dfco2, dfch4], ignore_index=True)
print(df)

          ECON_SECTOR                     SUBSECTOR STATE  GHG     Y1990  \
0         Agriculture  Agricultural Soil Management    CA  N2O  1.017179   
1         Agriculture  Agricultural Soil Management    CA  N2O  0.193156   
2         Agriculture  Agricultural Soil Management    CA  N2O  4.502394   
3         Agriculture  Agricultural Soil Management    CA  N2O  2.781138   
4         Agriculture  Agricultural Soil Management    CO  N2O  0.254721   
...               ...                           ...   ...  ...       ...   
20723  Transportation        Fossil Fuel Combustion    WI  CH4 -0.002664   
20724  Transportation        Fossil Fuel Combustion    WV  CH4 -0.000476   
20725  Transportation        Fossil Fuel Combustion    WY  CH4 -0.000150   
20726  Transportation        Fossil Fuel Combustion    AK  CH4 -0.001638   
20727  Transportation        Fossil Fuel Combustion    AL  CH4 -0.006979   

          Y1991     Y1992     Y1993     Y1994     Y1995  ...     Y2012  \
0      1.0007

In [ ]:
#Encoding
encoder = LabelEncoder()
for col in df.columns[:4]:
    df[col] = encoder.fit_transform(df[col])
print(df)
print(df.shape)

       ECON_SECTOR  SUBSECTOR  STATE  GHG     Y1990     Y1991     Y1992  \
0                0          0      5    2  1.017179  1.000724  0.996830   
1                0          0      5    2  0.193156  0.198694  0.210681   
2                0          0      5    2  4.502394  4.273469  4.211588   
3                0          0      5    2  2.781138  2.579990  2.690314   
4                0          0      6    2  0.254721  0.206407  0.236174   
...            ...        ...    ...  ...       ...       ...       ...   
20723            6          9     57    0 -0.002664 -0.002787 -0.002880   
20724            6          9     58    0 -0.000476 -0.000521 -0.000541   
20725            6          9     59    0 -0.000150 -0.000203 -0.000211   
20726            6          9      0    0 -0.001638 -0.001648 -0.001697   
20727            6          9      1    0 -0.006979 -0.007089 -0.007087   

          Y1993     Y1994     Y1995  ...     Y2012     Y2013     Y2014  \
0      1.129317  0.992245

In [ ]:
print(df.iloc[:,4:])

          Y1990     Y1991     Y1992     Y1993     Y1994     Y1995     Y1996  \
0      1.017179  1.000724  0.996830  1.129317  0.992245  1.164554  1.295379   
1      0.193156  0.198694  0.210681  0.204771  0.183397  0.214843  0.204504   
2      4.502394  4.273469  4.211588  4.351497  4.664195  4.578858  5.096083   
3      2.781138  2.579990  2.690314  2.710979  2.559154  2.748094  2.691244   
4      0.254721  0.206407  0.236174  0.220155  0.197366  0.351979  0.265394   
...         ...       ...       ...       ...       ...       ...       ...   
20723 -0.002664 -0.002787 -0.002880 -0.002835 -0.003282 -0.003114 -0.002333   
20724 -0.000476 -0.000521 -0.000541 -0.000530 -0.000767 -0.000852 -0.000605   
20725 -0.000150 -0.000203 -0.000211 -0.000207 -0.000186 -0.000196 -0.000229   
20726 -0.001638 -0.001648 -0.001697 -0.000811 -0.000919 -0.001432 -0.001167   
20727 -0.006979 -0.007089 -0.007087 -0.007946 -0.008434 -0.008338 -0.008366   

          Y1997     Y1998     Y1999  ...     Y2012 

In [ ]:
#Normalization and Feature Scaling
scaler = MinMaxScaler()
df.dropna()
features = df.iloc[:,:4].values
targets = df.iloc[:,4:].values
features = scaler.fit_transform(features)
print(len(features))
print(targets.shape)

20728
(20728, 32)


In [ ]:
#Combining Features and Targets
dat = np.hstack((features,targets))
print(dat.shape)

(20728, 36)


In [ ]:
#Fixing random number seed to reproduce results
np.random.seed(42)

In [ ]:
train_dat, test_dat = train_test_split(dat,test_size=0.20)
print(train_dat.shape)
n_inp = 4
train_gen = TimeseriesGenerator(train_dat,train_dat[:,-1],length=n_inp,batch_size=1)
test_gen = TimeseriesGenerator(test_dat,test_dat[:,-1],length=n_inp,batch_size=1)
print(train_gen)
print(test_gen)

(16582, 36)


In [ ]:
#Neural Network
model = Sequential()
#First LSTM layer with 20% dropout regularisation
model.add(LSTM(100, return_sequences=True, input_shape=(n_inp, train_dat.shape[1])))
model.add(Dropout(0.2))

#Second LSTM layer
model.add(LSTM(units=95, return_sequences=True))
model.add(Dropout(0.2))

#Third LSTM layer
model.add(LSTM(units=90, return_sequences=True))
model.add(Dropout(0.2))

#Output layer
model.add(Dense(units=1))

#Compile the LSTM
model.compile(optimizer='Adam',loss='mean_squared_error')

#Fit
model.fit(train_gen,epochs=50,batch_size=64,validation_data=train_gen)


Epoch 1/50
16578/16578 [==============================] - 465s 25ms/step - loss: 10.6610 - val_loss: 10.6464
Epoch 2/50
16578/16578 [==============================] - 411s 25ms/step - loss: 10.6508 - val_loss: 10.6535
Epoch 3/50
16578/16578 [==============================] - 350s 21ms/step - loss: 10.6544 - val_loss: 10.6434
Epoch 4/50
16578/16578 [==============================] - 408s 25ms/step - loss: 10.6552 - val_loss: 10.6536
Epoch 5/50
16578/16578 [==============================] - 413s 25ms/step - loss: 10.6546 - val_loss: 10.6498
Epoch 6/50
16578/16578 [==============================] - 411s 25ms/step - loss: 10.6605 - val_loss: 10.6438
Epoch 7/50
16578/16578 [==============================] - 351s 21ms/step - loss: 10.6603 - val_loss: 10.6416
Epoch 8/50
16578/16578 [==============================] - 353s 21ms/step - loss: 10.6636 - val_loss: 10.6612
Epoch 9/50
16578/16578 [==============================] - 356s 21ms/step - loss: 10.6648 - val_loss: 10.6441
Epoch 10/50
16578/1

In [ ]:
#Prediction
predicted_dat = model.predict(train_gen)
print(predicted_dat)
print(predicted_dat.shape)

16578/16578 [==============================] - 105s 6ms/step
[[[0.42315727]
  [0.44713336]
  [0.40512508]
  [0.42956978]]

 [[0.4247983 ]
  [0.45049125]
  [0.4068591 ]
  [0.43557346]]

 [[0.42079085]
  [0.44260514]
  [0.4091518 ]
  [0.43586814]]

 ...

 [[0.41982555]
  [0.44488966]
  [0.40554637]
  [0.42928797]]

 [[0.4282539 ]
  [0.4517265 ]
  [0.40615064]
  [0.4362265 ]]

 [[0.42427546]
  [0.44622695]
  [0.4113351 ]
  [0.439215  ]]]
(16578, 4, 1)


In [ ]:
from google.colab import drive
drive.mount('/content/mydrive')

Mounted at /content/mydrive


In [ ]:

model_save_path = '/content/mydrive/MyDrive/Colab Notebooks/mfc_g2.h5'
model.save(model_save_path)
